<a href="https://colab.research.google.com/github/PARTHIBAN-007/LLM-FineTuning/blob/main/llama_3_2_3B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

In [36]:
from huggingface_hub import notebook_login
notebook_login()

In [16]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None

model , tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True
)

==((====))==  Unsloth 2025.3.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [17]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj","k_proj","v_proj","o_proj",
                      "gate_proj","up_proj","down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    loftq_config = None,
)

In [18]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)



In [21]:
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}

from datasets import load_dataset
dataset = load_dataset("mlabonne/FineTome-100k", split="train")

from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [22]:
dataset[25]["conversations"]

[{'content': 'Write Python code to solve the task:\nYou are given a string of characters, or numbers. Find the minimum number of characters to be inserted into the string in order to obtain a palindrome. \nA palindrome is a word, phrase, number, or other sequence of symbols or elements that reads the same forward or reversed. \n\nFor example, the string abcbd can be transformed into a palindrome ("dabcbad" or "adbcbda"). However, inserting fewer than 2 characters will not produce a palindrome.\n\nInput Format:\n\nFirst line contains test cases and second line contains an integer \'n\' specifying the length of the string, where 3 ≤ n ≤ 20\nSecond line contains a string of length n. \n\nNote:\n\nUpper-case and lower-case characters are considered as different. Elements of the string are either English alphabets or numerals.\n\nOutput Format:\n\nOne line containing the minimum number of insertions required to make the string a palindrome\n\nSAMPLE INPUT\n2\n5\nnitin\n7\naabbaab\n\nSAMPLE 

In [23]:
dataset[25]["text"]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWrite Python code to solve the task:\nYou are given a string of characters, or numbers. Find the minimum number of characters to be inserted into the string in order to obtain a palindrome. \nA palindrome is a word, phrase, number, or other sequence of symbols or elements that reads the same forward or reversed. \n\nFor example, the string abcbd can be transformed into a palindrome ("dabcbad" or "adbcbda"). However, inserting fewer than 2 characters will not produce a palindrome.\n\nInput Format:\n\nFirst line contains test cases and second line contains an integer \'n\' specifying the length of the string, where 3 ≤ n ≤ 20\nSecond line contains a string of length n. \n\nNote:\n\nUpper-case and lower-case characters are considered as different. Elements of the string are either English alphabets or numer

In [26]:
from trl import SFTTrainer
from transformers import TrainingArguments , DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

training_args = TrainingArguments(
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    warmup_steps = 5,
    max_steps = 60,
    learning_rate = 1e-5,
    fp16 = not is_bfloat16_supported(),
    bf16 = is_bfloat16_supported(),
    logging_steps = 1,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type=  "linear",
    seed = 2503,
    output_dir = "ouputs",

)
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer ,
    train_dataset = dataset ,
    max_seq_length = 2048,
    dataset_text_field = "text",
    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer),
    packing = False, # training speed at 5x
    dataset_num_proc = 2,
    args = training_args,

)

Unsloth: We found double BOS tokens - we shall remove one automatically.


In [29]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer ,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [30]:
tokenizer.decode(trainer.train_dataset[25]["input_ids"])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWrite Python code to solve the task:\nYou are given a string of characters, or numbers. Find the minimum number of characters to be inserted into the string in order to obtain a palindrome. \nA palindrome is a word, phrase, number, or other sequence of symbols or elements that reads the same forward or reversed. \n\nFor example, the string abcbd can be transformed into a palindrome ("dabcbad" or "adbcbda"). However, inserting fewer than 2 characters will not produce a palindrome.\n\nInput Format:\n\nFirst line contains test cases and second line contains an integer \'n\' specifying the length of the string, where 3 ≤ n ≤ 20\nSecond line contains a string of length n. \n\nNote:\n\nUpper-case and lower-case characters are considered as different. Elements of the string are either English alphabets or numer

In [31]:
space = tokenizer(" ",add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x== -100 else x for x in trainer.train_dataset[25]["labels"]])

"                                                                                                                                                                                                                                                           Step 1: Input the number of test cases.\nStep 2: For each test case, input the length of the string and the string itself.\nStep 3: Initialize a variable 'insertions' to keep track of the number of insertions required to make the string a palindrome.\nStep 4: Check if the string is already a palindrome by comparing it with its reverse. If it is, set 'insertions' to 0.\nStep 5: If the string is not a palindrome, iterate through the string from the beginning and end simultaneously.\nStep 6: If the characters at the current positions are the same, move to the next positions.\nStep 7: If the characters are different, increment 'insertions' and compare the next characters from both ends.\nStep 8: Repeat steps 6 and 7 until the two pointers mee

In [32]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 2k23aids43 (unknown01-knowledge-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,0.935900
2,1.115300
3,0.770200
4,0.653000
5,0.851600
6,1.165900
7,1.039200
8,1.087300
9,0.810400
10,0.931600


In [34]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer ,
    chat_template = "llama-3.1",
)

FastLanguageModel.for_inference(model)

messages = [
    {
        "role":"user",
        "content" : "Continue the fibonnaci seuqence: 1,1,2,3,5,8"
    }
]

inputs  = tokenizer.apply_chat_template(
    messages ,
    tokenizer = True,
    add_generation_prompt = True ,
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs ,max_new_tokens = 64 ,use_cache = True,
                         temperature = 0.7 , min_p = 0.1)
tokenizer.batch_decode(outputs)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nContinue the fibonnaci seuqence: 1,1,2,3,5,8<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe Fibonacci sequence is a series of numbers in which each number is the sum of the two preceding numbers, starting from 1 and 1.\n\nThe sequence you provided is correct up to 8. Here's the next few numbers in the sequence:\n\n1, 1, 2, 3, 5,"]

In [35]:
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

To continue the Fibonacci sequence, we add the last two numbers:

1. Start with the given sequence: 1, 1, 2, 3, 5, 8
2. Calculate the next number by adding the last two numbers (8 + 5 = 13)
3. Calculate the next number by adding the last two numbers (13 + 8 = 21)

The Fibonacci sequence continues as follows:
13, 21, 34<|eot_id|>


In [37]:
model.save_pretrained("llama-3.2-3B")
tokenizer.save_pretrained("llama-3.2-3B")
model.push_to_hub("Parthiban007/llama-3.2-3B")
tokenizer.push_to_hub("Parthiban007/llama-3.2-3B")

README.md:   0%|          | 0.00/617 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/97.3M [00:00<?, ?B/s]

Saved model to https://huggingface.co/Parthiban007/llama-3.2-3B


  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]